# Using Markdown with OpenTURNS

In this example, we use the Markdown format to pretty-print
results from an uncertainty quantification study.
We first show how the Markdown function from the IPython module
works.
Then we show how this can be used within a uncertainty quantification
study.

## References

- https://github.com/openturns/openturns/pull/2415

In [1]:
import tabulate
import openturns as ot
from openturns.usecases import stressed_beam

In [2]:
# Table
data = []
n = 5
for i in range(n):
    data.append((i, 1 + i))
tabulate.tabulate(data, tablefmt="html", headers=["Index", "Value"])

Index,Value
0,1
1,2
2,3
3,4
4,5


## Format a reliability study

In this section, we format the results of a reliability study.
Before doing this, we must see that the library already provides
pretty-print features which can be accessed through various
objects.
The most simple pretty-print feature is given by the
`Sample`.


In [3]:
data = ot.Sample(5, 3)
data

,v0,v1,v2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0



This can be convenient to format any result that can be
stored within a `Sample`, i.e. any data set that
has the shape of an array of floats.
There are, however, situations where this framework is not
general enough.
For example, we may want to print strings, integers,
lists of strings or list of integers (e.g. :class:`~openturns.Indices`)
and these data types cannot be stored in a `Sample`.
In the next example, we want to print an interval: this cannot
be done easily using the `Sample` class.
We could, of course, print the lower and upper bounds
into two different columns of the array, but this is less
straightforward to understand as an interval.

The next function performs a reliability study based on the
stressed beam.
Given the number of outer loops, the maximum coefficient of variation
and the block size, the function returns the result of the probability
simulation algorithm.
The function uses a Monte-Carlo experiment.



In [4]:
def computeReliabilityResult(maximumOuterSamplingSize, maximumCoV, blockSize=1):
    sm = stressed_beam.AxialStressedBeam()
    limitStateFunction = sm.model
    inputRandomVector = ot.RandomVector(sm.distribution)
    outputRandomVector = ot.CompositeRandomVector(limitStateFunction, inputRandomVector)
    myEvent = ot.ThresholdEvent(outputRandomVector, ot.Less(), 0.0)
    experiment = ot.MonteCarloExperiment()
    algoMC = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
    algoMC.setMaximumOuterSampling(maximumOuterSamplingSize)
    algoMC.setBlockSize(blockSize)
    algoMC.setMaximumCoefficientOfVariation(maximumCoV)
    algoMC.run()
    psaResult = algoMC.getResult()
    return psaResult

Use the function.


In [5]:
maximumOuterSamplingSize = 100
maximumCoV = 0.0
psaResult = computeReliabilityResult(maximumOuterSamplingSize, maximumCoV)
psaResult

probabilityEstimate=2.000000e-02 varianceEstimate=1.960000e-04 standard deviation=1.40e-02 coefficient of variation=7.00e-01 confidenceLength(0.95)=5.49e-02 outerSampling=100 blockSize=1

We increase the sample size and compute the
probability estimate and the corresponding confidence interval.
The output of this algorithm is a `markdown` string which contains
the results inside a Markdown table.

In [7]:
sampleSizeList = []
numberOfRepetitions = 10
sampleSizeFactor = 2
alpha = 0.05
maximumOuterSamplingSize = 2
data = []
for i in range(numberOfRepetitions):
    maximumOuterSamplingSize *= sampleSizeFactor
    psaResult = computeReliabilityResult(maximumOuterSamplingSize, maximumCoV)
    probability = psaResult.getProbabilityEstimate()
    pflen = psaResult.getConfidenceLength(1 - alpha)
    pfLower = probability - pflen / 2
    pfUpper = probability + pflen / 2
    data.append(
        (
            maximumOuterSamplingSize,
            probability,
            f"{pfLower:.3e}, {pfUpper:.3e}",
        )
    )
tabulate.tabulate(
    data,
    tablefmt="html",
    headers=["n", "Pf", "95%  C.I."],
)

n,Pf,95% C.I.
4,0.25,"-1.743e-01, 6.743e-01"
8,0,"0.000e+00, 0.000e+00"
16,0.0625,"-5.611e-02, 1.811e-01"
32,0,"0.000e+00, 0.000e+00"
64,0.0625,"3.196e-03, 1.218e-01"
128,0.0546875,"1.530e-02, 9.408e-02"
256,0.0234375,"4.905e-03, 4.197e-02"
512,0.0214844,"8.925e-03, 3.404e-02"
1024,0.0341797,"2.305e-02, 4.531e-02"
2048,0.0253906,"1.858e-02, 3.220e-02"
